In [1]:
import dask, concurrent.futures, time, warnings, os, re, pickle
from osgeo import gdal
import os
import requests as r
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio as rio
import rioxarray as riox
import time
import xarray as xr
from urllib.request import urlopen
from xml.etree.ElementTree import parse,fromstring
from pandas import to_datetime
from rasterio.crs import CRS
from datetime import datetime, timedelta
from netrc import netrc
from pyproj import Proj
from src.hls_funcs import fetch
from src.hls_funcs.masks import mask_hls, shp2mask, bolton_mask
from src.hls_funcs.indices import ndvi_func
from src.hls_funcs.smooth import smooth_xr, despike_ts_xr
import cartopy.crs as ccrs
from rasterio.plot import show
from src.hls_funcs.predict import pred_bm, pred_bm_se, pred_cov
import dask.diagnostics
import matplotlib.pyplot as plt

In [2]:
t00 = time.time()
os.chdir('/project/cper_neon_aop/hls_nrt/')

In [3]:
"""
Get the server base url
"""
from jupyter_server import serverapp
from jupyter_server.utils import url_path_join
from pathlib import Path
import re
import requests
 
try:
    jupServer = [x for x in serverapp.list_running_servers()][0]
except IndexError:
    jupServer = {'base_url': '/node/ceres19-compute-98-eth.scinet.local/17710/'}

In [4]:
prefix = 'redtop' 
#prefix = 'cper'

if prefix == 'cper': 
    aoi_f = os.path.join('data/ground/cper_pastures_2017_clip.shp')
    df_aoi = gpd.read_file(aoi_f)
    subunit_name_old = 'Past_Name_'
    subunit_name = 'Pasture'
elif prefix == 'redtop':
    from src.utils.convert import kmz_to_shp
    df_aoi = kmz_to_shp('data/ground/RedTop_Boundary.kmz', 'data/ground/')
    df_aoi = df_aoi.to_crs(epsg=32613)
    subunit_name_old = None
    subunit_name = None

outDIR = '/90daydata/cper_neon_aop/hls_nrt/' + prefix

mod_bm = pickle.load(open('src/models/CPER_HLS_to_VOR_biomass_model_lr_simp.pk', 'rb'))
mod_cov = pickle.load(open('src/models/CPER_HLS_to_LPI_cover_pls_binned_model.pk', 'rb'))

/project/cper_neon_aop/conda_envs/hls_cog_env/lib/python3.9/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator PLSRegression from version 0.24.1 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
yr = 2021

In [6]:
print(yr)
t0 = time.time()
cluster_loc = 'hpc'
t1 = time.time()
if cluster_loc == 'local':
    print('   setting up Local cluster...')
    from dask.distributed import LocalCluster, Client
    import dask
    aws=False
    fetch.setup_env(aws=aws)
    cluster = LocalCluster(n_workers=8, threads_per_worker=2)
    client = Client(cluster)
    display(client)
elif cluster_loc == 'coiled':
    import coiled
    aws=True
    fetch.setup_env(aws=aws)
    s3_cred = fetch.setup_netrc(creds=['spkearney', '1mrChamu'], aws=aws)
    coiled.create_software_environment(
    name="hls_cog_coiled",
    conda="hls_cog_coiled_env.yaml")
    cluster = coiled.Cluster(
        name="hls_cog_coiled",
        software="kearney-sp/hls_cog_coiled",
        n_workers=5,
        worker_cpu=2,
        scheduler_cpu=2,
        backend_options={"region": "us-west-2"},
        environ=dict(GDAL_DISABLE_READDIR_ON_OPEN='FALSE', 
                   #AWS_NO_SIGN_REQUEST='YES',
                   GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
                   GDAL_SWATH_SIZE='200000000',
                   VSI_CURL_CACHE_SIZE='200000000',
                   CPL_VSIL_CURL_ALLOWED_EXTENSIONS='TIF',
                   GDAL_HTTP_UNSAFESSL='YES',
                   GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                   GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'),
                   AWS_REGION='us-west-2',
                   AWS_SECRET_ACCESS_KEY=s3_cred['secretAccessKey'],
                   AWS_ACCESS_KEY_ID=s3_cred['accessKeyId'],
                   AWS_SESSION_TOKEN=s3_cred['sessionToken'])
    )
    client = Client(cluster)
    display(client)
elif cluster_loc == 'hpc':
    from dask.distributed import LocalCluster, Client
    import dask_jobqueue as jq
    import dask
    print('   setting up moderate cluster on HPC...')
    aws=False
    fetch.setup_env(aws=aws)
    dask.config.set({'distributed.dashboard.link': jupServer['base_url'] + 'proxy/{port}/status'})
    partition='short',#'short','debug', 'mem', 'mem-low',
    num_processes = 4
    num_threads_per_processes = 2
    mem = 2.5*num_processes*num_threads_per_processes
    n_cores_per_job = num_processes*num_threads_per_processes
    clust = jq.SLURMCluster(queue=partition,
                            processes=num_processes,
                            cores=n_cores_per_job,
                            memory=str(mem)+'GB',
                            interface='ib0',
                            #interface='enp24s0f0',
                            local_directory='$TMPDIR',
                            death_timeout=30,
                            walltime='02:00:00',
                            job_extra=["--output=/dev/null","--error=/dev/null"])
    client=Client(clust)
    #Scale Cluster 
    num_jobs=16
    clust.scale(jobs=num_jobs)
    try:
        client.wait_for_workers(n_workers=num_jobs*num_processes, timeout=60)
    except dask.distributed.TimeoutError as e:
        print(str(num_jobs*num_processes) + ' workers not available. Continuing with available workers.')
        #print(e)
        pass
    display(client)
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

if not os.path.exists(outDIR):
    os.mkdir(outDIR)

if subunit_name_old is not None:
    df_aoi = df_aoi.rename(columns={subunit_name_old: subunit_name})
if subunit_name is not None:
    df_aoi = df_aoi.dissolve(by=subunit_name).reset_index()

start_date = str(yr - 1) + "-11-01"
end_date = str(yr + 1) + "-03-01"

# set the date range for analysis
date_rng = pd.date_range(start=start_date, end=end_date)
date_rng = date_rng[date_rng <= datetime.today()]

/project/cper_neon_aop/conda_envs/hls_cog_env/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


2021
   setting up moderate cluster on HPC...


<Client: 'tcp://10.1.9.127:37588' processes=40 threads=80, memory=186.40 GiB>

...completed in 13.0 secs
total elasped time: 0.22 mins



In [7]:
print('   fetching data...')
t1 = time.time()
# fetch the COG data from the AWS bucket
data_dict = {'date_range': [str(date_rng.min().date()), str(date_rng.max().date())]}
hls_ds = fetch.get_hls(hls_data=data_dict,
                       bbox=df_aoi.total_bounds, 
                       proj_epsg=df_aoi.crs.to_epsg(),
                       lim=500,
                       aws=aws)
hls_ds = hls_ds.where(hls_ds['eo:cloud_cover'] != None, drop=True)
hls_ds = hls_ds.where(hls_ds['eo:cloud_cover'] < 90, drop=True)
#hls_ds = hls_ds.loc[dict(x=slice(517587.0, 527283.0), y=slice(4524402.0, 4514699.0))].chunk({'y': -1,
#                                                                                             'x': -1,
#                                                                                             'time':1})
hls_mask = mask_hls(hls_ds['FMASK'])
idx_load_try = 0
max_load_try = 5
while idx_load_try < max_load_try:
    try:
        hls_ds = hls_ds.where(hls_mask == 0).load()
        idx_load_try = max_load_try
    except RuntimeError as e:            
        if e.args[0] == 'Set changed size during iteration':
            print('Warning: error with cluster set size. Restarting cluster and retrying ' + \
              str(idx_load_try+1) + ' of ' + str(max_load_try))
            clust.close()
            client.close()
            clust = jq.SLURMCluster(queue=partition,
                            processes=num_processes,
                            cores=n_cores_per_job,
                            memory=str(mem)+'GB',
                            interface='ib0',
                            #interface='enp24s0f0',
                            local_directory='$TMPDIR',
                            death_timeout=30,
                            walltime='02:00:00',
                            job_extra=["--output=/dev/null","--error=/dev/null"])
            client=Client(clust)
            #Scale Cluster 
            num_jobs=8
            clust.scale(jobs=num_jobs)
            try:
                client.wait_for_workers(n_workers=num_jobs*num_processes, timeout=60)
            except dask.distributed.TimeoutError as e:
                print(str(num_jobs*num_processes) + ' workers not available. Continuing with available workers.')
                #print(e)
                pass
            display(client)
        else:
            print('Warning: error connecting to lpdaac. Retrying ' + str(idx_load_try+1) + ' of ' + str(max_load_try))
        idx_load_try += 1
    except rio.errors.RasterioIOError as e:
        print('Warning: error loading data. Retrying ' + str(idx_load_try+1) + ' of ' + str(max_load_try))
        idx_load_try += 1

#hls_ds['time'] = hls_ds['time'].dt.date
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

   fetching data...
...completed in 921.0 secs
total elasped time: 15.57 mins



In [8]:
print(hls_ds)

<xarray.Dataset>
Dimensions:         (time: 251, y: 905, x: 777)
Coordinates:
  * time            (time) object 2020-11-04 2020-11-04 ... 2022-02-23
    id              (time) <U34 'HLS.S30.T13SEC.2020309T174519.v2.0' ... 'HLS...
  * x               (x) float64 5.505e+05 5.505e+05 ... 5.738e+05 5.738e+05
  * y               (y) float64 4.209e+06 4.209e+06 ... 4.182e+06 4.182e+06
    eo:cloud_cover  (time) int64 0 1 53 18 31 17 4 15 ... 84 33 2 2 63 14 66 38
    epsg            int64 32613
Data variables:
    BLUE            (time, y, x) float64 868.0 848.0 ... 1.071e+03 1.087e+03
    GREEN           (time, y, x) float64 1.216e+03 1.192e+03 ... 1.504e+03
    RED             (time, y, x) float64 1.574e+03 1.54e+03 ... 1.888e+03
    NIR1            (time, y, x) float64 2.198e+03 2.13e+03 ... 2.478e+03
    SWIR1           (time, y, x) float64 3.223e+03 3.223e+03 ... 3.571e+03
    SWIR2           (time, y, x) float64 2.595e+03 2.604e+03 ... 3.071e+03
    FMASK           (time, y, x) float64

In [9]:
if cluster_loc == 'hpc':
    print('   closing local HPC cluster and scaling up to new cluster...')
    t1 = time.time()
    client.close()
    clust.close()
    partition='short',#'short','debug', 'mem', 'mem-low',
    num_processes = 4
    num_threads_per_processes = 2
    mem = 8*num_processes*num_threads_per_processes#*1.25
    n_cores_per_job = num_processes*num_threads_per_processes
    clust = jq.SLURMCluster(queue=partition,
                            processes=num_processes,
                            cores=n_cores_per_job,
                            memory=str(mem)+'GB',
                            interface='ib0',
                            #interface='enp24s0f0',
                            local_directory='$TMPDIR',
                            death_timeout=30,
                            walltime='02:00:00',
                            job_extra=["--output=/dev/null","--error=/dev/null"])
    client=Client(clust)
    #Scale Cluster 
    num_jobs=16
    clust.scale(jobs=num_jobs)
    try:
        client.wait_for_workers(n_workers=num_jobs*num_processes, timeout=60)
    except dask.distributed.TimeoutError as e:
        print(str(num_jobs*num_processes) + ' workers not available. Continuing with available workers.')
        #print(e)
        pass
    display(client)
    #num_jobs=16
    #clust.scale(jobs=num_jobs)
    #client.wait_for_workers(n_workers=num_jobs*num_processes, timeout=60)
    print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
    print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

   closing local HPC cluster and scaling up to new cluster...


<Client: 'tcp://10.1.9.127:41282' processes=40 threads=80, memory=596.00 GiB>

...completed in 21.0 secs
total elasped time: 16.68 mins



In [9]:
hls_ds = hls_ds.reset_coords(drop=True)
print(hls_ds)

<xarray.Dataset>
Dimensions:  (time: 251, y: 905, x: 777)
Coordinates:
  * time     (time) object 2020-11-04 2020-11-04 ... 2022-02-23 2022-02-23
  * x        (x) float64 5.505e+05 5.505e+05 5.506e+05 ... 5.738e+05 5.738e+05
  * y        (y) float64 4.209e+06 4.209e+06 4.209e+06 ... 4.182e+06 4.182e+06
Data variables:
    BLUE     (time, y, x) float64 868.0 848.0 842.0 ... 1.071e+03 1.087e+03
    GREEN    (time, y, x) float64 1.216e+03 1.192e+03 ... 1.476e+03 1.504e+03
    RED      (time, y, x) float64 1.574e+03 1.54e+03 ... 1.852e+03 1.888e+03
    NIR1     (time, y, x) float64 2.198e+03 2.13e+03 ... 2.454e+03 2.478e+03
    SWIR1    (time, y, x) float64 3.223e+03 3.223e+03 ... 3.578e+03 3.571e+03
    SWIR2    (time, y, x) float64 2.595e+03 2.604e+03 ... 3.1e+03 3.071e+03
    FMASK    (time, y, x) float64 64.0 64.0 64.0 64.0 ... 128.0 128.0 128.0
Attributes:
    spec:        RasterSpec(epsg=32613, bounds=(550500, 4181640, 573810, 4208...
    crs:         epsg:32613
    transform:   | 30

In [10]:
if len(np.unique(hls_ds.values)) < len(hls_ds.time.values):
    print('    reducing along id dimension to single observation for each date')
    #hls_ds = hls_ds.chunk({'time': 1, 'y': 50, 'x': 50})
    t1 = time.time()
    hls_ds = hls_ds.groupby('time').mean().compute()
    print(hls_ds)
    print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
    print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

    reducing along id dimension to single observation for each date
<xarray.Dataset>
Dimensions:  (time: 125, y: 905, x: 777)
Coordinates:
  * x        (x) float64 5.505e+05 5.505e+05 5.506e+05 ... 5.738e+05 5.738e+05
  * y        (y) float64 4.209e+06 4.209e+06 4.209e+06 ... 4.182e+06 4.182e+06
  * time     (time) object 2020-11-04 2020-11-09 ... 2022-02-23 2022-02-27
Data variables:
    BLUE     (time, y, x) float64 868.0 848.0 842.0 841.0 ... 908.0 926.0 980.0
    GREEN    (time, y, x) float64 1.216e+03 1.192e+03 ... 1.292e+03 1.374e+03
    RED      (time, y, x) float64 1.574e+03 1.54e+03 ... 1.672e+03 1.777e+03
    NIR1     (time, y, x) float64 2.198e+03 2.13e+03 ... 2.239e+03 2.306e+03
    SWIR1    (time, y, x) float64 3.223e+03 3.223e+03 ... 3.388e+03 3.403e+03
    SWIR2    (time, y, x) float64 2.595e+03 2.604e+03 ... 2.883e+03 2.873e+03
    FMASK    (time, y, x) float64 64.0 64.0 64.0 64.0 ... 64.0 64.0 64.0 64.0
...completed in 26.0 secs
total elasped time: 16.01 mins



In [15]:
hls_ds['time'] = pd.to_datetime(hls_ds.time)

In [16]:
if not os.path.exists(os.path.join(outDIR, prefix + '_hls_tmp/')):
        os.mkdir(os.path.join(outDIR, prefix + '_hls_tmp/'))
hls_ds.to_netcdf(os.path.join(outDIR,
                              prefix + 
                              '_hls_tmp/hls_ds_' + 
                              str(yr) +'_' + 
                              datetime.now().strftime('%Y%m%d') + '.nc'))

In [17]:
print(hls_ds)

<xarray.Dataset>
Dimensions:  (time: 125, y: 905, x: 777)
Coordinates:
  * x        (x) float64 5.505e+05 5.505e+05 5.506e+05 ... 5.738e+05 5.738e+05
  * y        (y) float64 4.209e+06 4.209e+06 4.209e+06 ... 4.182e+06 4.182e+06
  * time     (time) datetime64[ns] 2020-11-04 2020-11-09 ... 2022-02-27
Data variables:
    BLUE     (time, y, x) float64 868.0 848.0 842.0 841.0 ... 908.0 926.0 980.0
    GREEN    (time, y, x) float64 1.216e+03 1.192e+03 ... 1.292e+03 1.374e+03
    RED      (time, y, x) float64 1.574e+03 1.54e+03 ... 1.672e+03 1.777e+03
    NIR1     (time, y, x) float64 2.198e+03 2.13e+03 ... 2.239e+03 2.306e+03
    SWIR1    (time, y, x) float64 3.223e+03 3.223e+03 ... 3.388e+03 3.403e+03
    SWIR2    (time, y, x) float64 2.595e+03 2.604e+03 ... 2.883e+03 2.873e+03
    FMASK    (time, y, x) float64 64.0 64.0 64.0 64.0 ... 64.0 64.0 64.0 64.0


In [18]:
    
print('   applying secondary cloud mask...')
hls_ds = hls_ds.chunk({'time': -1, 'y': 50, 'x': 50})
t1 = time.time()
hls_bolton_mask = bolton_mask(hls_ds)
hls_ds = hls_ds.where(hls_bolton_mask == 0)#.compute()
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

print('   computing NDVI for available dates...')
hls_ds = hls_ds.chunk({'time': 20, 'y': 50, 'x': 50})
t1 = time.time()
# create ndvi array
xr_ndvi = ndvi_func(hls_ds)#.compute()
xr_ndvi.name = 'NDVI'
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

print('   computing biomass for available dates...')
t1 = time.time()
# create biomass array
xr_bm = hls_ds.map_blocks(pred_bm, template=hls_ds['NIR1'],
                          kwargs=dict(model=mod_bm)).where(hls_ds['NIR1'].notnull())#.compute()
xr_bm.name = 'Biomass'
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

print('   computing biomass SE for available dates...')
t1 = time.time()
# create biomass array
xr_bm_se = hls_ds.map_blocks(pred_bm_se, template=hls_ds['NIR1'],
                          kwargs=dict(model=mod_bm)).where(hls_ds['NIR1'].notnull())#.compute()
xr_bm_se.name = 'Biomass_SE'
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

print('   computing cover for available dates...')
t1 = time.time()
hls_ds = hls_ds.chunk({'time': 5, 'y': -1, 'x': -1})
# create cover array
xr_cov = pred_cov(hls_ds, model=mod_cov).where(hls_ds['NIR1'].notnull())#.compute()
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

print('   removing dates with > 75% cloud cover...')
t1 = time.time()
partial_mask = (xr_ndvi.count(dim=['y', 'x'])/np.product(xr_ndvi.shape[1:])) > 0.25
xr_ndvi = xr_ndvi.where(partial_mask, drop=True)
xr_bm = xr_bm.where(partial_mask, drop=True)
xr_bm_se = xr_bm_se.where(partial_mask, drop=True)
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

   applying secondary cloud mask...
...completed in 0.0 secs
total elasped time: 22.77 mins

   computing NDVI for available dates...
...completed in 0.0 secs
total elasped time: 22.77 mins

   computing biomass for available dates...
...completed in 1.0 secs
total elasped time: 22.8 mins

   computing biomass SE for available dates...
...completed in 1.0 secs
total elasped time: 22.82 mins

   computing cover for available dates...
...completed in 5.0 secs
total elasped time: 22.9 mins

   removing dates with > 75% cloud cover...
...completed in 238.0 secs
total elasped time: 26.86 mins



In [19]:
ds_vars = xr.merge([xr_ndvi, xr_bm, xr_bm_se, xr_cov])
ds_vars['time'] = pd.to_datetime(ds_vars['time'])

/project/cper_neon_aop/conda_envs/hls_cog_env/lib/python3.9/site-packages/dask/array/core.py:4441: PerformanceWarning: Increasing number of chunks by factor of 16
  result = blockwise(
/project/cper_neon_aop/conda_envs/hls_cog_env/lib/python3.9/site-packages/dask/array/core.py:4441: PerformanceWarning: Increasing number of chunks by factor of 16
  result = blockwise(
/project/cper_neon_aop/conda_envs/hls_cog_env/lib/python3.9/site-packages/dask/array/core.py:4441: PerformanceWarning: Increasing number of chunks by factor of 16
  result = blockwise(


In [20]:
print(ds_vars)

<xarray.Dataset>
Dimensions:     (time: 125, x: 777, y: 905)
Coordinates:
  * time        (time) datetime64[ns] 2020-11-04 2020-11-09 ... 2022-02-27
  * x           (x) float64 5.505e+05 5.505e+05 ... 5.738e+05 5.738e+05
  * y           (y) float64 4.209e+06 4.209e+06 ... 4.182e+06 4.182e+06
Data variables:
    NDVI        (time, y, x) float64 dask.array<chunksize=(21, 50, 50), meta=np.ndarray>
    Biomass     (time, y, x) float64 dask.array<chunksize=(21, 50, 50), meta=np.ndarray>
    Biomass_SE  (time, y, x) float64 dask.array<chunksize=(21, 50, 50), meta=np.ndarray>
    BARE        (time, y, x) float32 dask.array<chunksize=(5, 905, 777), meta=np.ndarray>
    SD          (time, y, x) float32 dask.array<chunksize=(5, 905, 777), meta=np.ndarray>
    GREEN       (time, y, x) float32 dask.array<chunksize=(5, 905, 777), meta=np.ndarray>
    LITT        (time, y, x) float32 dask.array<chunksize=(5, 905, 777), meta=np.ndarray>


In [21]:
print('   creating daily template for output...')
t1 = time.time()
# create an output Dataset template with all dates
dat_out = xr.Dataset(coords={'y': ds_vars.y,
                             'x': ds_vars.x,
                             'time': [x for x in date_rng if x not in ds_vars['time'].values]})#.chunk({'time': -1, 'y': 50, 'x': 50})

# assign the time coordinate to the output
dat_out['time'] = dat_out['time']

# create empty arrays for filling variable values
dat_out = dat_out.assign(NDVI=(['time', 'y', 'x'],
                               np.zeros((dat_out.dims['time'], 
                                         dat_out.dims['y'], 
                                         dat_out.dims['x'])) * np.nan),
                         Biomass=(['time', 'y', 'x'],
                               np.zeros((dat_out.dims['time'], 
                                         dat_out.dims['y'], 
                                         dat_out.dims['x'])) * np.nan),
                         Biomass_SE=(['time', 'y', 'x'],
                               np.zeros((dat_out.dims['time'], 
                                         dat_out.dims['y'], 
                                         dat_out.dims['x'])) * np.nan),
                         BARE=(['time', 'y', 'x'],
                               np.zeros((dat_out.dims['time'], 
                                         dat_out.dims['y'], 
                                         dat_out.dims['x'])) * np.nan),
                         SD=(['time', 'y', 'x'],
                               np.zeros((dat_out.dims['time'], 
                                         dat_out.dims['y'], 
                                         dat_out.dims['x'])) * np.nan),
                         GREEN=(['time', 'y', 'x'],
                               np.zeros((dat_out.dims['time'], 
                                         dat_out.dims['y'], 
                                         dat_out.dims['x'])) * np.nan),
                         LITT=(['time', 'y', 'x'],
                               np.zeros((dat_out.dims['time'], 
                                         dat_out.dims['y'], 
                                         dat_out.dims['x'])) * np.nan)
                         )

   creating daily template for output...


In [22]:
dat_out_ds = xr.concat([dat_out, ds_vars], dim='time')
dat_out_ds = dat_out_ds.sortby('time')

/project/cper_neon_aop/conda_envs/hls_cog_env/lib/python3.9/site-packages/xarray/core/indexing.py:1227: PerformanceWarning: Slicing with an out-of-order index is generating 25 times more chunks
  return self.array[key]
/project/cper_neon_aop/conda_envs/hls_cog_env/lib/python3.9/site-packages/xarray/core/indexing.py:1227: PerformanceWarning: Slicing with an out-of-order index is generating 25 times more chunks
  return self.array[key]
/project/cper_neon_aop/conda_envs/hls_cog_env/lib/python3.9/site-packages/xarray/core/indexing.py:1227: PerformanceWarning: Slicing with an out-of-order index is generating 25 times more chunks
  return self.array[key]


In [ ]:
# combine the available NDVI data with the output Dataset template
#dat_out_ds.attrs = hls_ds.attrs
dat_out_ds['NDVI_despike'] = despike_ts_xr(dat_out_ds['NDVI'].chunk({'y': 5, 'x': 5, 'time':-1}), 
                                           dat_thresh=0.07, 
                                           mask_outliers=False,
                                           iters=2,
                                           dims=['time'])
dat_out_ds['NDVI_smooth'] = smooth_xr(dat_out_ds['NDVI_despike'], 
                                      dims=['time'], 
                                      kwargs={'double': True, 'limit': 91})
dat_out_ds['Biomass_despike'] = despike_ts_xr(dat_out_ds['Biomass'].chunk({'y': 5, 'x': 5, 'time':-1}),
                                              dat_thresh=150, 
                                              mask_outliers=True,
                                              z_thresh=5.0,
                                              iters=2,
                                              dims=['time'])
dat_out_ds['Biomass_smooth'] = smooth_xr(dat_out_ds['Biomass_despike'], 
                                         dims=['time'],
                                         kwargs={'double': True, 'limit': 91})
dat_out_ds['Biomass_SE_despike'] = despike_ts_xr(dat_out_ds['Biomass_SE'].chunk({'y': 5, 'x': 5, 'time':-1}),
                                              dat_thresh=dat_out_ds['Biomass_SE'].std('time').mean().values * 3.0, 
                                              mask_outliers=True,
                                              z_thresh=5.0,
                                              iters=2,
                                              dims=['time'])
dat_out_ds['Biomass_SE_smooth'] = smooth_xr(dat_out_ds['Biomass_SE_despike'], 
                                         dims=['time'],
                                         kwargs={'double': True, 'limit': 91})

for c in ['BARE', 'SD', 'GREEN', 'LITT']:
    dat_out_ds[c + '_despike'] = despike_ts_xr(dat_out_ds[c].chunk({'y': 5, 'x': 5, 'time':-1}),
                                              dat_thresh=0.3, 
                                              mask_outliers=True,
                                              z_thresh=5.0,
                                              iters=2,
                                              dims=['time'])
    dat_out_ds[c + '_smooth'] = smooth_xr(dat_out_ds[c + '_despike'], 
                                         dims=['time'],
                                         kwargs={'double': True, 'limit': 91})
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

t1 = time.time()
print('   interpolating daily NDVI...')
dat_out_ds['NDVI_smooth'] = dat_out_ds['NDVI_smooth'].compute()
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

show(dat_out_ds['NDVI_smooth'].isel(time=150), vmin=0.10, vmax=0.35)

t1 = time.time()
print('   interpolating daily biomass...')
dat_out_ds['Biomass_smooth'] = dat_out_ds['Biomass_smooth'].compute()
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

show(dat_out_ds['Biomass_smooth'].isel(time=150), vmin=250, vmax=1000)

t1 = time.time()
print('   interpolating daily biomass SE...')
dat_out_ds['Biomass_SE_smooth'] = dat_out_ds['Biomass_SE_smooth'].compute()
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')
show(dat_out_ds['Biomass_SE_smooth'].isel(time=150))

for c in ['BARE', 'SD', 'GREEN', 'LITT']:
    print(c)
    t1 = time.time()
    print('   interpolating daily ' + c + '...')
    dat_out_ds[c + '_smooth'] = dat_out_ds[c + '_smooth'].compute()
    print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
    print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')
    show(dat_out_ds[c + '_smooth'].isel(time=150), vmin=0.05, vmax=0.60)

In [ ]:
t1 = time.time()
print('   subsetting data to ' + str(yr) + ' only and saving to disk as geotiffs...')
yr_mask = dat_out_ds['time'].astype(np.datetime64).dt.year == yr
out_profile = {'driver': 'GTiff',
               'dtype': 'float32',
               'nodata': -9999.,
               'width': len(dat_out_ds.x),
               'height': len(dat_out_ds.y),
               'count': len(dat_out_ds.sel(time=yr_mask).time),
               'crs': CRS.from_dict(init='epsg:32613'),
               'transform': rio.Affine(30.0, 0.0, np.min(dat_out_ds.x).data,
                                       0.0, -30.0, np.max(dat_out_ds.y).data),
               'tiled': False}

if not os.path.exists(os.path.join(outDIR, 'hls_ndvi/')):
    os.mkdir(os.path.join(outDIR, 'hls_ndvi/'))
if not os.path.exists(os.path.join(outDIR, 'hls_biomass/')):
    os.mkdir(os.path.join(outDIR, 'hls_biomass/'))
if not os.path.exists(os.path.join(outDIR, 'hls_cover/')):
    os.mkdir(os.path.join(outDIR, 'hls_cover/'))

with rio.open(os.path.join(outDIR, 'hls_ndvi/' + prefix + '_hls_ndvi_' + str(yr) + '.tif'), 
              'w',
              **out_profile) as dst:
    dst.write(dat_out_ds.sel(time=yr_mask)['NDVI_smooth'].astype('float32'))

with rio.open(os.path.join(outDIR, 'hls_biomass/' + prefix + '_hls_bm_' + str(yr) + '.tif'), 
              'w',
              **out_profile) as dst:
    dst.write(dat_out_ds.sel(time=yr_mask)['Biomass_smooth'].astype('float32'))

with rio.open(os.path.join(outDIR, 'hls_biomass/' + prefix + '_hls_bm_se_' + str(yr) + '.tif'), 
              'w',
              **out_profile) as dst:
    dst.write(dat_out_ds.sel(time=yr_mask)['Biomass_SE_smooth'].astype('float32'))

for c in ['BARE', 'SD', 'GREEN', 'LITT']:
    with rio.open(os.path.join(outDIR, 'hls_cover/' + prefix + '_hls_' + c + '_' + str(yr) + '.tif'), 
                  'w',
                  **out_profile) as dst:
        dst.write(dat_out_ds.sel(time=yr_mask)[c + '_smooth'].astype('float32'))
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

In [ ]:
    
#dat_out_ds.sel(
#    time=yr_mask)['NDVI_smooth'].astype('float32').rio.to_raster(
#    os.path.join(outDIR, 'hls_ndvi/' + prefix + '_hls_ndvi_' + str(yr) + '.tif'))
#dat_out_ds.sel(
#    time=yr_mask)['Biomass_smooth'].astype('float32').rio.to_raster(
#    os.path.join(outDIR, 'hls_biomass/' + prefix + '_hls_bm_' + str(yr) + '.tif'))
#dat_out_ds.sel(
#    time=yr_mask)['Biomass_SE_smooth'].astype('float32').rio.to_raster(
#    os.path.join(outDIR, 'hls_biomass/' + prefix + '_hls_bm_se_' + str(yr) + '.tif'))


if subunit_name is not None:
    t1 = time.time()
    print('   adding subunit data as a spatial coordinate...')
    aoi_info = df_aoi[[subunit_name, 'geometry']].reset_index(drop=True).reset_index().rename(columns={'index': 'id'})
    past_dict = {row.id+1: row.Pasture for _, row in aoi_info.iterrows()}
    past_dict[0] = 'UNK'
    aoi_mask_shp = [(row.geometry, row.id+1) for _, row in aoi_info.iterrows()]
    aoi_mask = shp2mask(shp=aoi_mask_shp, 
                         transform=rio.Affine(30.0, 0.0, dat_out_ds.x.min(), 0.0, -30.0, dat_out_ds.y.max()), 
                         outshape=dat_out_ds['NDVI'].shape[1:], 
                         xr_object=dat_out_ds['NDVI'])
    past_mask = np.array([past_dict[i] for i in aoi_mask.values.flatten()]).reshape(aoi_mask.shape)
    dat_out_ds = dat_out_ds.assign(Pasture=(['y', 'x'], past_mask)).chunk({'y': 50, 'x': 50})
    dat_out_ds = dat_out_ds.set_coords(subunit_name)
    print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
    print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

    show(dat_out_ds['NDVI_smooth'].isel(time=200).where(
        dat_out_ds[subunit_name].isin(list(df_aoi[subunit_name].unique()[:5]))))
    
    if not os.path.exists(os.path.join(outDIR, prefix + '_hls_means/')):
        os.mkdir(os.path.join(outDIR, prefix + '_hls_means/'))
    t1 = time.time()
    print('   computing daily averages for individual subunits and saving to disk...')
    df_out_aoi = dat_out_ds.sel(
        time=yr_mask).mean(dim=['y', 'x']).to_dataframe()

    df_out_aoi[subunit_name] = 'AOI'
    df_out_aoi = df_out_aoi.drop(columns='epsg').reset_index()

    df_out_pasts = dat_out_ds.sel(
        time=yr_mask).groupby(subunit_name).mean(dim='stacked_y_x').to_dataframe()

    df_out_pasts = df_out_pasts.drop(columns='epsg').reset_index()

    df_out_all = pd.concat([df_out_aoi, df_out_pasts])
    df_out_all.rename(columns={'time': 'Date'}, inplace=True)
    
    print('    rounding outputs')
    df_out_all.loc[:,
                   [c for c in df_out_all.columns if 'NDVI' in c]] = df_out_all.loc[:,
                                                                                    [c for c in df_out_all.columns if 'NDVI' in c]].apply(
        lambda x: round(x, 4))
    df_out_all.loc[:,
                   [c for c in df_out_all.columns if 'Biomass' in c and not 'SE' in c]] = df_out_all.loc[:,
                                                                                    [c for c in df_out_all.columns if 'Biomass' in c]].apply(
        lambda x: round(x, 0))

    df_out_all[df_out_all[subunit_name] != 'UNK'].sort_values(
        [subunit_name, 'Date']).to_csv(
        os.path.join(outDIR, prefix + '_hls_means/' + prefix + '_hls_means_' + str(yr) + '.csv'),
        index=False)
    print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
    print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')


    print('checking output dataframe NDVI data')
    ax1 = df_out_all[df_out_all[subunit_name]=='AOI'].plot('Date', 'NDVI_smooth', c='orange')
    df_out_all[df_out_all[subunit_name]=='AOI'].plot.scatter('Date', 'NDVI', ax=ax1)
    df_out_all[df_out_all[subunit_name]==df_aoi[subunit_name].unique()[0]].plot('Date', 
                                                                                'NDVI_smooth', 
                                                                                c='red',
                                                                                ax=ax1)
    df_out_all[df_out_all[subunit_name]==df_aoi[subunit_name].unique()[1]].plot('Date',
                                                                                'NDVI_smooth',
                                                                                c='blue', 
                                                                                ax=ax1)
    plt.show()
    plt.close()

    print('checking output dataframe biomass data')
    ax2 = df_out_all[df_out_all[subunit_name]=='AOI'].plot('Date', 'Biomass_smooth', c='orange')
    df_out_all[df_out_all[subunit_name]=='AOI'].plot.scatter('Date', 'Biomass', ax=ax2)
    df_out_all[df_out_all[subunit_name]==df_aoi[subunit_name].unique()[0]].plot('Date', 
                                                                                'Biomass_smooth',
                                                                                c='red',
                                                                                ax=ax2)
    df_out_all[df_out_all[subunit_name]==df_aoi[subunit_name].unique()[1]].plot('Date', 
                                                                                'Biomass_smooth',
                                                                                c='blue', 
                                                                                ax=ax2)
    plt.show()
    plt.close()
    
    print('checking output dataframe biomass SE data')
    ax2 = df_out_all[df_out_all[subunit_name]=='AOI'].plot('Date', 'Biomass_SE_smooth', c='orange')
    df_out_all[df_out_all[subunit_name]=='AOI'].plot.scatter('Date', 'Biomass_SE', ax=ax2)
    df_out_all[df_out_all[subunit_name]==df_aoi[subunit_name].unique()[0]].plot('Date', 
                                                                                'Biomass_SE_smooth',
                                                                                c='red',
                                                                                ax=ax2)
    df_out_all[df_out_all[subunit_name]==df_aoi[subunit_name].unique()[1]].plot('Date', 
                                                                                'Biomass_SE_smooth',
                                                                                c='blue', 
                                                                                ax=ax2)
    plt.show()
    plt.close()

print('all analysis complete for year ' + str(yr) + '. Finished in ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

In [ ]:
dask.distributed.TimeoutError

In [ ]:
dat_out_ds['NDVI_smooth'].isel(time=200)#.where(dat_out_ds[subunit_name].isin(df_aoi[subunit_name].unique()[:5]))

In [ ]:
dat_out_ds['NDVI_smooth'].isel(time=200).where(dat_out_ds[subunit_name].isin(list(df_aoi[subunit_name].unique()[:5])))

In [ ]:
list(df_aoi[subunit_name].unique()[:5])